<a href="https://colab.research.google.com/github/nikhil-nikss/DeepLearning/blob/main/RESNET.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### RESNET Implementation

In [17]:
# importing important packages

from tensorflow.keras.applications.resnet50 import ResNet50,preprocess_input
import tensorflow.keras as keras
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import optimizers
import tensorflow as tf
from keras.utils import np_utils
from keras.models import load_model,Sequential
from keras.datasets import cifar10
from keras.preprocessing import image
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import cv2

In [44]:
# Loading pretrained model

conv_base = ResNet50(weights = 'imagenet',include_top = False,input_shape = (256,256,3))

In [29]:
# checking Model summary

conv_base.summary()

Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 200, 200, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 206, 206, 3)  0           input_3[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 100, 100, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 100, 100, 64) 256         conv1_conv[0][0]                 
___________________________________________________________________________________________

In [45]:
# Creating train and test 

(x_train,y_train),(x_test,y_test) = cifar10.load_data()

In [46]:
# converting to grey scale

x_train = x_train/255.0
x_test = x_test/255.0

In [47]:
# converting to categorical

y_train = np_utils.to_categorical(y_train,10)
y_test = np_utils.to_categorical(y_test,10)

In [48]:
# checking shape of train and test

print(x_train.shape)
print(x_test.shape)

(50000, 32, 32, 3)
(10000, 32, 32, 3)


In [49]:
# creating model

model = models.Sequential()
model.add(layers.UpSampling2D((2,2)))
model.add(layers.UpSampling2D((2,2)))
model.add(layers.UpSampling2D((2,2)))

model.add(conv_base) # adding pretrained model between my custom model architecture and this technique is called transfer learning technique

model.add(layers.Flatten())
model.add(layers.BatchNormalization())
model.add(layers.Dense(128,activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.BatchNormalization())
model.add(layers.Dense(64,activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.BatchNormalization())
model.add(layers.Dense(10,activation='softmax'))

In [50]:
# compiling the model

model.compile(optimizer= optimizers.RMSprop(learning_rate=2e-5),loss='binary_crossentropy',metrics = ['acc'])

In [52]:
# Fitting the model

history = model.fit(x_train,y_train,epochs = 2,batch_size = 20,validation_data= (x_test,y_test))

Epoch 1/2
2500/2500 [==============================] - 394s 158ms/step - loss: 0.5642 - acc: 0.6439 - val_loss: 0.3708 - val_acc: 0.8568
Epoch 2/2
2500/2500 [==============================] - 394s 158ms/step - loss: 0.4428 - acc: 0.7492 - val_loss: 0.2648 - val_acc: 0.9010


In [56]:
# Saving model file

model.save('resnet_model.h5')

In [59]:
# testing on new image

img = cv2.imread("test.jpg")
dim = (32,32)
img = cv2.resize(img, dim, interpolation=cv2.INTER_AREA)
x = image.img_to_array(img)
x = np.expand_dims(x,axis = 0)
x = preprocess_input(x)
preds = model.predict(x)

In [60]:
preds

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]], dtype=float32)